In [3]:
import Pude_training_loop.loss_functions_torch as loss_functions
import Pude_training_loop.pude_utils as pude_utils
import Pude_training_loop.model_training as model_training
import monocular_stereo_matching.stereo_dataset_loader as data_loader
from monocular_stereo_matching.stereo_pair_gen import Stereo_Pair_Generator
import torch
from PIL import Image
import pickle
import numpy as np
import os

import monocular_stereo_matching.stereo_eval as stereo_eval

device = "cuda" if torch.cuda.is_available() else "cpu"
# model_path = "new_unimatch_model_pude_AdamW_beta_60_600_alphas_4_0_4_epochs_3_lr_1e-6.pth"
model_path ="new_unimatch_model_Adamw_betas_60_600,_alphas_4_0_4_checkpoint_epoch_0_finished.pth"

# model_path = "new_unimatch_model_cosine_scheduler_AdamW_betas_100_50,_alphas_4_0-4_4_lr_5e-6_epochs_3.pth"
# model_path = "new_unimatch_model_cosine_scheduler_AdamW_betas_60_60,_alphas_4_0-4_8_lr_5e-6_epochs_3.pth"
# model_path = "new_unimatch_model_cosine_scheduler_AdamW_betas_200_100,_alphas_4_0-4_4_lr_5e-6_epochs_3.pth"
datasets = ["rock_garden1", "rock_garden2"]
# datasets = ["canyon2"]
# datasets = ["rock_garden2"]
# model_type = "pude"
# new_model_type = "new_pude"
# model_type = "unimatch"
# new_model_type = "unimatch"

# model_paths = [f"new_unimatch_model_edge_aware_loss_cosine_scheduler_AdamW_betas_2_0_0.1_0.5_lr_1e-4_epochs_3.pth",
#                f"new_unimatch_model_edge_aware_loss_cosine_scheduler_AdamW_betas_2_0_0_0.5_lr_1e-4_epochs_3.pth", 
#                 f"new_unimatch_model_edge_aware_loss_cosine_scheduler_AdamW_betas_2_0_0_0_lr_1e-4_epochs_3.pth"]
# model_paths_nums = [f"new_combined_2_0_0.1_0.5_epochs_3_lr_1e-4",
#                     f"new_combined_2_0_0_0.5_epochs_3_lr_1e-4", 
#                     f"new_combined_2_0_0_0_epochs_3_lr_1e-4"]  


# model_type = "unimatch"
# new_model_type = "unimatch"

# model_paths = [f"new_unimatch_model_edge_aware_loss_cosine_scheduler_AdamW_betas_2_0_0.1_0.5_lr_1e-4_epochs_3.pth"]
# model_paths_nums = [f"new_combined_2_0_0.1_0.5_epochs_3_lr_1e-4"]

model_type = "unimatch_small"
new_model_type = "unimatch_small"

model_paths = ["new_unimatch_small_model_FLsea_Canyon1_new_pude_edge_aware_loss_cosine_scheduler_AdamW_betas_2_0_0.08_0.4_lr_0.0001_epochs_3.pth",
               "new_unimatch_small_model_FLsea_Canyon1_new_pude_edge_aware_loss_cosine_scheduler_AdamW_betas_2_0_0.4_0.4_lr_0.0001_epochs_3.pth",
               "new_unimatch_small_model_FLsea_Canyon1_new_pude_edge_aware_loss_cosine_scheduler_AdamW_betas_2_0_0.1_0.5_lr_0.0001_epochs_3.pth"]
model_paths_nums = ["unimatch_small_2_0_0.08_0.4",
                    "unimatch_small_2_0_0.4_0.4",
                    "unimatch_small_2_0_0.1_0.5"]



for i in range(0, len(model_paths)):
    # model_path = f"new_unimatch_model_edge_aware_loss_cosine_scheduler_AdamW_betas_1_40_4_alphas_1_0.1_4_lr_5e-06_epoch_{i}.pth"
    # model_path  = f"new_pude_model_Adam_betas_1_0-002,_alphas_1_0-1_1.pth"
    # model_path = f"new_unimatch_model_edge_aware_loss_cosine_scheduler_AdamW_betas_4_0_0.1_0.5_lr_0.0001_epoch_{i}.pth"
    # model_path =  f"new_unimatch_model_edge_aware_loss_cosine_scheduler_AdamW_betas_100_80_10_alphas_4_0-4_4_lr_5e-6_epochs_{i}.pth"
    # model_path =  "new_unimatch_model_cosine_scheduler_AdamW_betas_300_400,_alphas_4_0-4_4_lr_5e-6_epochs_3.pth"
    # model_path = f"new_unimatch_model_edge_aware_loss_cosine_scheduler_AdamW_betas_4_0_0.1_0.5_lr_1e-4_epoch_{i}.pth"
    # model_path_nums = f"new_combined_4_0_0.1_0.5_epoch_{i}_lr_1e-4"  
    
    model_path = model_paths[i]
    model_path_nums = model_paths_nums[i]

    # org_unimatch_model, org_unimatch_image_processor = model_training.get_model_image_processor_pair(model_name=model_type, model_path=model_training.models[model_type], device=device)
    new_unimatch_model, new_unimatch_image_processor = model_training.get_model_image_processor_pair(model_name=new_model_type, model_path=model_path, device=device)
    total_images = 100
    for dataset_name in datasets:
        # create the results folder if it doesnt exist
        if not os.path.exists(f"Results/FLSea/{dataset_name}/new_{model_type}_{model_path_nums}/images"):
            os.makedirs(f"Results/FLSea/{dataset_name}/new_{model_type}_{model_path_nums}/images")
            os.makedirs(f"Results/FLSea/{dataset_name}/new_{model_type}_{model_path_nums}/rplots")
            os.makedirs(f"Results/FLSea/{dataset_name}/combined_{model_type}_{model_path_nums}/images")
        if not os.path.exists(f"Results/FLSea/{dataset_name}/org_{model_type}/images"):
            os.makedirs(f"Results/FLSea/{dataset_name}/org_{model_type}/images")
            os.makedirs(f"Results/FLSea/{dataset_name}/org_{model_type}/rplots")

       
        dataset_names = [dataset_name]
        save_ground_truth = False

        dataset_loader = data_loader.DatasetLoader(total_num_pairs=total_images, dataset_names=dataset_names) # Initialize dataset loader with default parameters
        stereo_pair_gen = Stereo_Pair_Generator(image_dim=model_training.default_image_dim)
        stereo_evaluator = stereo_eval.StereoEval()


        new_unimatch_results_path = f"Results/FLSea/{dataset_name}/new_{model_type}_{model_path_nums}/images"
        org_unimatch_results_path = f"Results/FLSea/{dataset_name}/org_{model_type}/images"
        combined_grid_path = f"Results/FLSea/{dataset_name}/combined_{model_type}_{model_path_nums}/images"
        new_unimatch_results_rplt_path = f"Results/FLSea/{dataset_name}/new_{model_type}_{model_path_nums}/rplots"
        org_unimatch_results_rplt_path = f"Results/FLSea/{dataset_name}/org_{model_type}/rplots"

        # seed the torch
        torch.manual_seed(model_training.seed)
        np.random.seed(model_training.seed)


        new_evals = np.zeros(len(stereo_evaluator.criteria_functions))
        # org_evals = np.zeros(len(stereo_evaluator.criteria_functions))
       

        # orig_unimatch_results = []

        def get_model_output(unimatch_model, unimatch_image_processor, input, ground_truth, valid_indices, device, rplot_path=None):
            unimatch_output = model_training.get_model_output(model=unimatch_model, 
                                                            image_processor=unimatch_image_processor, 
                                                            raw_image=input, 
                                                            device=device, requires_grad=False)
            unimatch_output = unimatch_output.cpu().detach().numpy().squeeze()
            ground_truth_image = pude_utils.get_depth_as_image(ground_truth)
            model_output_image = pude_utils.get_depth_as_image(unimatch_output)
            diff_image = np.zeros_like(ground_truth)
            diff_image[valid_indices] = np.abs(ground_truth[valid_indices]-unimatch_output[valid_indices])
            abs_difference_image = pude_utils.get_depth_as_image(diff_image)
            if model_type == "unimatch" or model_type == "unimatch_small":
                left_image = Image.fromarray(input['image1'])
            elif model_type == "pude" or model_type == "depth_anything"  :
                left_image = Image.fromarray(input)
            image_grid = pude_utils.make_image_grid_title(images=[left_image, ground_truth_image, model_output_image, abs_difference_image], 
                                                titles=["Left Image", "Ground Truth", "Model Output", "Absolute Difference"],
                                                rows=1, cols=4)
            evals = stereo_evaluator.evaluate(ground_truth[valid_indices], unimatch_output[valid_indices],rplot_path=rplot_path)
            return unimatch_output, model_output_image, evals, image_grid


        for i in range(total_images):
            left_image, right_image, ground_truth, valid_indices = dataset_loader[i]
            # left_image = torch.tensor(left_image)
            # right_image = torch.tensor(right_image)

            if model_type == "unimatch" or model_type == "unimatch_small":
                unimatch_input = {"image1": left_image, "image2": right_image}
            elif model_type == "pude" or model_type == "depth_anything":
                unimatch_input = left_image
            if save_ground_truth:
                org_rplot_path = f"{org_unimatch_results_rplt_path}/image_{i}.png"
            else:
                org_rplot_path = None
            new_rplot_path = f"{new_unimatch_results_rplt_path}/image_{i}.png"
            # new_rplot_path = None
            # org_output, org_output_image, _org_evals, org_image_grid = get_model_output(org_unimatch_model, 
            #                                                                         org_unimatch_image_processor, 
            #                                                                         unimatch_input, 
            #                                                                         ground_truth,
            #                                                                             valid_indices, device, rplot_path=org_rplot_path)
            new_output, new_output_image, _new_evals, new_image_grid = get_model_output(new_unimatch_model, 
                                                                                    new_unimatch_image_processor, 
                                                                                    unimatch_input, 
                                                                                    ground_truth,
                                                                                    valid_indices,device, rplot_path=new_rplot_path )

            new_evals+=_new_evals
            # org_evals+=_org_evals
            # orig_unimatch_results.append(org_output)

            # diff_image = pude_utils.get_depth_as_image(np.abs(org_output-new_output))
            # diff_grid = pude_utils.make_image_grid_title(images=[Image.fromarray(left_image), org_output_image, new_output_image, diff_image],
            #                                                 titles=["Left Image", f"Original {model_type} Output", f"New {model_type} Output", "Absolute Difference"],
            #                                                 rows=1, cols=4)
            # combined_grid = pude_utils.make_image_grid_title(images=[org_image_grid, new_image_grid, diff_grid],
            #                                                 titles=[f"Original {model_type}", f"New {model_type}", "Difference"],
            #                                                 rows=3, cols=1)
            # if save_ground_truth:
            #     org_image_grid.save(f"{org_unimatch_results_path}/image_{i}.png")
            new_image_grid.save(f"{new_unimatch_results_path}/image_{i}.png")
            # combined_grid.save(f"{combined_grid_path}/image_{i}.png")


        new_evals/=total_images
        # org_evals/=total_images

        print(f"-------------------{dataset_name} Results-------------------")
        # print("original unimatch: ")
        # print(stereo_evaluator.criteria_functions.keys())
        # print(org_evals)
        print(f"{model_path[:-4]}: ")
        print(stereo_evaluator.criteria_functions.keys())
        print(new_evals)
        print("------------------------------------------------------------")
        # if save_ground_truth:
        #     with open(f"Results/FLSea/{dataset_name}/orig_{model_type}_results.pkl", "wb") as f:
        #         pickle.dump(orig_unimatch_results, f)

-------------------rock_garden1 Results-------------------
new_unimatch_small_model_FLsea_Canyon1_new_pude_edge_aware_loss_cosine_scheduler_AdamW_betas_2_0_0.08_0.4_lr_0.0001_epochs_3: 
dict_keys(['abs_rel_error', 'squared_rel_error', 'rmse', 'rmse_log', 'silog', 'accuracy_1.05', 'accuracy_1.05^2', 'accuracy_1.05^3', 'r_value'])
[8.33919913e+00 6.85313350e+07 4.17173058e+00 9.74654263e-01
 7.75176425e-01 8.81878963e-02 1.70458699e-01 2.44999102e-01
 8.16104244e-01]
------------------------------------------------------------
-------------------rock_garden2 Results-------------------
new_unimatch_small_model_FLsea_Canyon1_new_pude_edge_aware_loss_cosine_scheduler_AdamW_betas_2_0_0.08_0.4_lr_0.0001_epochs_3: 
dict_keys(['abs_rel_error', 'squared_rel_error', 'rmse', 'rmse_log', 'silog', 'accuracy_1.05', 'accuracy_1.05^2', 'accuracy_1.05^3', 'r_value'])
[9.24047588e-01 2.54803311e+04 3.62587758e+00 6.16864619e-01
 5.23665452e-01 1.12434281e-01 2.19077789e-01 3.18391871e-01
 8.13296538e-01]